## Algoritmos Genéticos

### Punto 1

Módelo para el siguiente problema: Suponga que usted es el jefe de gobierno y está interesado en que pasen los proyectos de su programa político. Sin embargo, en el congreso conformado por 5 partidos, no es fácil su tránsito, por lo que debe repartir el poder, conformado por ministerios y otras agencias del gobierno, con base en la representación de cada partido. Cada entidad estatal tiene un peso de poder, que es el que se debe distribuir. Suponga que hay 50 curules, distribuya aleatoriamente, con una distribución no informe entre los 5 partidos esas curules. Defina una lista de 50 entidades y asígneles aleatoriamente un peso político de 1 a 100 puntos. Cree una matriz de poder para repartir ese poder, usando AGs.

In [ ]:
pip install deap

In [3]:
import random
import numpy as np

# Número de partidos y curules
n_partidos = 5
curules_totales = 50

# Distribuir curules entre los partidos (distribución no uniforme)
curules_por_partido = np.random.multinomial(curules_totales, [0.1, 0.2, 0.3, 0.25, 0.15])
print("Curules por partido:", curules_por_partido)

# Generar entidades con pesos políticos
entidades = [f"Entidad_{i}" for i in range(curules_totales)]
pesos_politicos = [random.randint(1, 100) for _ in entidades]

print("\nEntidades y sus pesos políticos:")
for entidad, peso in zip(entidades, pesos_politicos):
    print(f"{entidad}: {peso}")

from deap import base, creator, tools, algorithms

# Crear la clase Fitness y el individuo
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar el error
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Definir la estructura del individuo
toolbox.register("attr_party", lambda: random.randint(0, n_partidos - 1))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_party, n=len(entidades))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluación: calcular el error de distribución de poder
def evaluar(individuo):
    # Sumar los pesos asignados a cada partido
    poder_partidos = [0] * n_partidos
    for entidad, partido in zip(entidades, individuo):
        poder_partidos[partido] += pesos_politicos[entidades.index(entidad)]
    
    # Error: diferencia absoluta entre poder y proporción de curules
    proporcion_esperada = [curules / curules_totales * sum(pesos_politicos) for curules in curules_por_partido]
    error = sum(abs(poder_partidos[i] - proporcion_esperada[i]) for i in range(n_partidos))
    return error,

toolbox.register("evaluate", evaluar)

# Operadores genéticos
toolbox.register("mate", tools.cxTwoPoint)  # Cruce de dos puntos
toolbox.register("mutate", tools.mutUniformInt, low=0, up=n_partidos - 1, indpb=0.2)  # Mutación
toolbox.register("select", tools.selTournament, tournsize=3)  # Selección por torneo

# Configurar parámetros del algoritmo genético
pop = toolbox.population(n=100)
n_generaciones = 50

# Ejecutar el AG
resultados, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_generaciones, verbose=True)


# Obtener el mejor individuo
mejor_individuo = tools.selBest(pop, k=1)[0]

# Mostrar la asignación final
poder_final = [0] * n_partidos
for entidad, partido in zip(entidades, mejor_individuo):
    poder_final[partido] += pesos_politicos[entidades.index(entidad)]

print("\nDistribución final del poder político por partido:")
for i in range(n_partidos):
    print(f"Partido {i + 1}: {poder_final[i]} puntos de poder")

print("\nCurules por partido (proporción esperada):", curules_por_partido)


Curules por partido: [ 3 10 15 14  8]

Entidades y sus pesos políticos:
Entidad_0: 81
Entidad_1: 15
Entidad_2: 78
Entidad_3: 77
Entidad_4: 67
Entidad_5: 35
Entidad_6: 16
Entidad_7: 73
Entidad_8: 86
Entidad_9: 76
Entidad_10: 70
Entidad_11: 86
Entidad_12: 24
Entidad_13: 49
Entidad_14: 34
Entidad_15: 5
Entidad_16: 13
Entidad_17: 8
Entidad_18: 59
Entidad_19: 61
Entidad_20: 20
Entidad_21: 39
Entidad_22: 19
Entidad_23: 5
Entidad_24: 54
Entidad_25: 53
Entidad_26: 13
Entidad_27: 74
Entidad_28: 29
Entidad_29: 21
Entidad_30: 76
Entidad_31: 38
Entidad_32: 27
Entidad_33: 92
Entidad_34: 95
Entidad_35: 38
Entidad_36: 100
Entidad_37: 94
Entidad_38: 11
Entidad_39: 99
Entidad_40: 53
Entidad_41: 91
Entidad_42: 69
Entidad_43: 89
Entidad_44: 99
Entidad_45: 77
Entidad_46: 74
Entidad_47: 90
Entidad_48: 52
Entidad_49: 73
gen	nevals
0  	100   
1  	74    
2  	87    
3  	80    
4  	80    
5  	74    
6  	80    
7  	70    
8  	78    
9  	73    
10 	83    
11 	68    
12 	75    
13 	56    
14 	75    
15 	62    
16 

## Punto 2:

In [5]:
import numpy as np
import random

# Parámetros del problema
capacities = [3, 6, 5, 4]  # Capacidad de cada planta
demands = [4, 3, 5, 3]     # Demanda de cada ciudad
transport_costs = np.array([
    [1, 4, 3, 6],
    [4, 1, 4, 5],
    [3, 4, 1, 4],
    [6, 5, 4, 1]
])
generation_costs = [680, 720, 660, 750]  # Costo por GW de cada planta

# Función de evaluación
def evaluate(individual):
    transport_cost = np.sum(transport_costs * individual)
    generation_cost = sum(generation_costs[i] * np.sum(individual[i, :]) for i in range(4))
    return transport_cost + generation_cost

# Validar y corregir una solución para que cumpla las restricciones
def validate_solution(individual):
    # Ajustar filas para no exceder la capacidad de las plantas
    for i in range(4):
        total_generated = np.sum(individual[i, :])
        if total_generated > capacities[i]:
            excess = total_generated - capacities[i]
            for j in range(4):
                if individual[i, j] > 0:
                    reduction = min(excess, individual[i, j])
                    individual[i, j] -= reduction
                    excess -= reduction
                    if excess <= 0:
                        break

    # Ajustar columnas para cumplir con la demanda de las ciudades
    for j in range(4):
        total_received = np.sum(individual[:, j])
        if total_received < demands[j]:
            deficit = demands[j] - total_received
            for i in range(4):
                available_capacity = capacities[i] - np.sum(individual[i, :])
                addition = min(deficit, available_capacity)
                individual[i, j] += addition
                deficit -= addition
                if deficit <= 0:
                    break
    return individual

# Generación inicial aleatoria respetando restricciones
def initialize_population(pop_size=50):
    population = []
    for _ in range(pop_size):
        individual = np.zeros((4, 4))
        for j in range(4):  # Distribuir la demanda de cada ciudad
            remaining_demand = demands[j]
            while remaining_demand > 0:
                i = random.randint(0, 3)  # Seleccionar una planta aleatoria
                available_capacity = capacities[i] - np.sum(individual[i, :])
                allocation = min(remaining_demand, available_capacity)
                individual[i, j] += allocation
                remaining_demand -= allocation
        population.append(validate_solution(individual))
    return population

# Selección de los mejores individuos
def select(population, fitness, num_parents=20):
    parents_indices = np.argsort(fitness)[:num_parents]
    return [population[i] for i in parents_indices]

# Crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(1, 3)
    child = np.vstack((parent1[:crossover_point], parent2[crossover_point:]))
    return validate_solution(child)

# Mutación
def mutate(individual, mutation_rate=0.1):
    for _ in range(int(4 * 4 * mutation_rate)):  # Aproximadamente un 10% de las celdas
        i, j = random.randint(0, 3), random.randint(0, 3)
        adjustment = random.uniform(-1, 1)
        individual[i, j] = max(0, individual[i, j] + adjustment)
    return validate_solution(individual)

# Algoritmo genético principal
def genetic_algorithm(iterations=100, pop_size=50):
    population = initialize_population(pop_size)
    for generation in range(iterations):
        fitness = [evaluate(ind) for ind in population]
        parents = select(population, fitness)
        new_population = parents[:]
        while len(new_population) < pop_size:
            p1, p2 = random.sample(parents, 2)
            child = mutate(crossover(p1, p2))
            new_population.append(child)
        population = new_population
    # Devolver el mejor individuo
    best_idx = np.argmin([evaluate(ind) for ind in population])
    return population[best_idx], evaluate(population[best_idx])

# Ejecutar
best_solution, best_cost = genetic_algorithm()
print("Mejor solución encontrada:")
print(best_solution)
print("Costo total:", best_cost)



Mejor solución encontrada:
[[3. 0. 0. 0.]
 [1. 3. 0. 2.]
 [0. 0. 5. 0.]
 [0. 0. 0. 1.]]
Costo total: 10436.0
